### Week 11
##### Iterative Processing for Generating SAS Code
(Sequential vs. Non-Sequential Processing)

[What macro variables cannot do](https://documentation.sas.com/?cdcId=pgmsascdc&cdcVersion=9.4_3.5&docsetId=mcrolref&docsetTarget=n1nks63d6g1rk2n1owajhi374c6w.htm&locale=en)

Macro variables are useful for simple text substitution. 

They cannot perform conditional operations, DO loops, and other more complex tasks. For this type of work, you must define a macro.

##### In the code cell below, we do the following:

* Define the macro.
* Declare index variable of the macro loop as a local macro variable.
* Specify the keyword parameters in the %macro statement.
* Invoke the macro.

##### Goal: To create 8 data sets, each containing 2 observations.

###### Steps: 

* Define the macro.

* Declare index variable of the macro loop as a local macro variable.

* Specify the keyword parameters in the %macro statement.

* Invoke the macro to create 8 separate data sets.

* Concatenate them.

* Generate listing.



In [1]:
options nocenter nodate nosource nonumber;
ods html close;
%macro runit (first=, last=);
      %local yr;
      %do yr=&first %to &last;
        data have_20%sysfunc(putn(&yr,z2.));
              exp=20%sysfunc(putn(&yr,z2.))/4;
       year=20%sysfunc(putn(&yr,z2.));
     run; 
       %end ;
%mend runit;
%runit(first=8, last=15)

data all_yrs;
 retain year;
   set Have:;
run;
proc print data=all_yrs noobs split='*';
label year= 'Survey Year'
      exp= 'Mean expenses*for treatment*on influenza*per person per year';
run;

Survey Year,Mean expensesfor treatmenton influenzaper person per year
2008,502.00
2009,502.25
2010,502.50
2011,502.75
2012,503.00
2013,503.25
2014,503.50
2015,503.75


In [4]:
options nonumber nocenter nodate symbolgen;
%macro modular (dsn);
   title  "sashelp.&dsn";
   proc print data=sashelp.&dsn (obs=5) noobs;  
   run;
%mend modular;
%modular(class)
%modular(iris)
%modular(retail)

Name,Sex,Age,Height,Weight
Alfred,M,14,69.0,112.5
Alice,F,13,56.5,84.0
Barbara,F,13,65.3,98.0
Carol,F,14,62.8,102.5
Henry,M,14,63.5,102.5
Species,SepalLength,SepalWidth,PetalLength,PetalWidth
Setosa,50,33,14,2
Setosa,46,34,14,3
Setosa,46,36,10,2
Setosa,51,33,17,5


In [ ]:
options nonumber nocenter nodate symbolgen;
%LET list = %str(sashelp.class| 
                 sashelp.iris| 
                 sashelp.retail);
/* Count # of values in the string */
%LET count=%sysfunc(countw(&list, %STR(|))); 
%macro loop;
 /* Loop through the total # of data sets */ 
 %local i;
 %do i = 1 %to &count;
   title  "%left(%unquote(%SCAN(&list, &i, %STR(|))))";
   proc print data=%scan(&list,&i, %str(|)) 
             (obs=5) noobs;  
run;
 %end;
%mend loop;
%loop


##### Macro with a list of values of a macro Variable (No keyword or positional parameters)

* The macro variable DSLIST contains the data set names.
* The %DO loop is used to step through the list with %DO loop index (&J) serving as the word counter.  

* The %SCAN is used to retrieve the &jth word (data set names).
* The call to macro prints observations from each of the three data sets.


In [ ]:
options nocenter nodate nonumber symbolgen;
%macro loop(dslist);    
     %local xcount i; 
     /* Count the # of values in the string */                                                                                                                                   
     %let xcount=%sysfunc(countw(&dslist, %STR(|))); 
     /* Loop through the total # of data sets */                                                                                         
     %do i = 1 %to &xcount; 
        title  "%left(%scan(&dslist,&i,%str(|)))"; 
        proc print data=%scan(&dslist,&i,%str(|))
                 (obs=5) noobs;
run;
%end;                                                                                            
%mend loop;                                            
%loop(%str(sashelp.class|sashelp.iris|sashelp.retail))


##### Macro with a list of macro Variables (Array of Macro Variables)

* There is an array of macro variables (DS1, DS2 and DS3).

* Each macro variable has a common prefix (DS) and also an iterative integer counter (suffix) on the end.

* The two ampersands (&&) in front of each of the macro variable name force the macro processor to scan the macro variable twice.  

    * The 1st iteration  resolves &&DS&i to &DS1 in the first scan 
      and &DS1 to CLASS in the second scan.
       
    * The 2nd iteration resolves &&DS&i to &DS2 in the first scan 
      and &DS2 to REVHUB2 in the second scan.
      
    * The 3rd iteration resolves &&DS&i to &DS3 in the first scan 
      and &DS3 to IRIS in the second scan.



In [ ]:
options nodate nonumber symbolgen;
%macro VList;
%local ds1 ds2 ds3 j;
%let ds1 = class;
%let ds2 = revhub2;
%let ds3 = iris;
%let dscount = 3;
%do j = 1 %to &dscount;
  title1 "%upcase(sashelp.&&ds&j)";
  proc print data=sashelp.&&ds&j (obs=3) noobs;
  run;
%end;
%mend VList;
%VList

##### Macro with a single macro variable that contains an array of elements (macro variable arrays)

* The macro variable DSLIST contains an array of data set names.
* The %DO loop is used to step through the list with %DO loop index (&J) serving as the word counter.  

* The %SCAN is used to retrieve the &jth data set of interest from the macro variable array (DSLIST).
* The call to macro prints observations from each of the three data sets.


In [ ]:
options nodate nonumber symbolgen;
%macro HList;
  %local dslist j;
  %let dslist = class revhub2 iris;
  %do j =1 %to %sysfunc(countw(&dslist));
   title1 "sashelp.%scan(&dslist,&j)";
   proc print data= sashelp.%scan(&dslist, &j)(obs=3) noobs;
   run;
  %end;
%mend HList;
%HList
